In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


In [2]:
from automl_libs import base_layer_utils, stacknet
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [4]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list, low, high):
    features_to_gen = []
    for function in function_list:
        for i in range(low, high+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, 
#                                        fe.cumulative_count, 
#                                        fe.reverse_cumulative_count, 
#                                        fe.variance, fe.count_std_over_mean], 
#                                       2, 2)# len(categorical_features))
# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count])
features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count], 2, 2)#len(categorical_features))

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DepTime'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'UniqueCarrier'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Origin'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Dest'],
  'function': <function automl_libs.feature_engineering.co

In [5]:
len(features_to_gen)

84

In [6]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project2/' 

In [7]:
from automl_app import logger_config
import logging
logger_config.config(project_path+'log_project2.log', file_loglevel=logging.INFO)

In [8]:
def gs_params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 30, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':10,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc' 
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.005, # if == lr_init, then no lr decay
            'batch_size': np.random.choice([32, 64, 128, 256, 512, 1024, 2048]),
            "pred_batch_size": 50000,
            'max_ep': 1,
            'patience': 1,
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'dense_units': 50,
            'num_dense_n_layers': 1,
            'drop_rate': 0.2,
            'combined_dense_n_layers': 1,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max'  # MUST HAVE
        }
    return params, seed

In [9]:
gs_params_gen('nn')

({'batch_size': 64,
  'cat_emb_outdim': 50,
  'combined_dense_n_layers': 1,
  'dense_units': 50,
  'drop_rate': 0.2,
  'ep_for_lr': 1,
  'lr_fin': 0.005,
  'lr_init': 0.01,
  'max_ep': 1,
  'mode': 'max',
  'monitor': 'val_auc',
  'nn_seed': 31774688,
  'num_dense_n_layers': 1,
  'patience': 1,
  'pred_batch_size': 50000},
 31774681)

In [10]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, gs_params_gen)

2018-07-02 15:24:29 | INFO    | automl_app.app.AlphaBoosting.__init__             | #22  | ==========NEW RUN==========
/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/app.py:139: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.df = pd.concat([self.train, self.test], ignore_index=True)
2018-07-02 15:24:29 | INFO    | automl_app.app.AlphaBoosting.__init__             | #65  | Run record file [./output/last_run_record.json] not found. Begin the first time run...
2018-07-02 15:24:29 | INFO    | automl_app.app.AlphaBoosting.__init__             | #96  | generate todo list
2018-07-02 15:24:29 | INFO    | automl_app.app.AlphaBoosting.__init__             | #112 | STAGE: FEATURE_ENGINEERING
2018-07-02 15:24:29 | INFO    | automl_app.app.AlphaBoosting.__init__             | 

[20]	cv_agg's auc: 0.721094 + 0.00127641


2018-07-02 15:25:05 | INFO    | automl_libs.grid_search._lgb_gs              | #89  | val_auc: 0.72812 (cv, no train_auc)
2018-07-02 15:25:05 | INFO    | automl_libs.grid_search._lgb_gs              | #109 | [do_preds] is True, generating predictions ...
2018-07-02 15:25:05 | INFO    | automl_libs.grid_search._lgb_gs              | #110 | Retrain model using best_round and all data...


[6]	training's auc: 0.713099
[12]	training's auc: 0.721388
[18]	training's auc: 0.729736
[24]	training's auc: 0.737361


2018-07-02 15:25:07 | INFO    | automl_libs.grid_search._lgb_gs              | #118 | Training done. Iteration: 30 | train_auc: 0.74383 | 176 features


...

2018-07-02 15:26:11 | INFO    | automl_libs.base_layer_utils.get_oof              | #436 | metric of fold 4: 0.7298033667113144
2018-07-02 15:26:11 | INFO    | automl_libs.base_layer_utils.get_oof              | #422 | processing fold 5 of 5...


No evaluation set, thus not possible to use early stopping. Please train with your best params.
[6]	training's auc: 0.710371
[12]	training's auc: 0.723025
[18]	training's auc: 0.73201
[24]	training's auc: 0.739547
[30]	training's auc: 0.747181


2018-07-02 15:26:12 | INFO    | automl_libs.base_layer_utils.get_oof              | #436 | metric of fold 5: 0.7257337704551687


No evaluation set, thus not possible to use early stopping. Please train with your best params.
[6]	training's auc: 0.713099
[12]	training's auc: 0.721388
[18]	training's auc: 0.729736
[24]	training's auc: 0.737361
[30]	training's auc: 0.743827


2018-07-02 15:26:14 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.add_score            | #292 | 9lfo__LGB_flight_data_ordinal_layer1 found in repo. Update score from 0 to 0.73195
2018-07-02 15:26:14 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.add_score            | #292 | Jk5W__LGB_flight_data_ordinal_layer1 found in repo. Update score from 0 to 0.72406
2018-07-02 15:26:14 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.add_score            | #292 | tBuQ__LGB_flight_data_ordinal_layer1 found in repo. Update score from 0 to 0.72812
2018-07-02 15:26:14 | DEBUG   | automl_libs.base_layer_utils.BaseLayerResultsRepo.__init__             | #233 | load StackNet saves from file
2018-07-02 15:26:14 | INFO    | automl_libs.stacknet.layer2               | #113 | All available layer1 model_data:
2018-07-02 15:26:14 | INFO    | automl_libs.stacknet.layer2               | #114 | [('9lfo__LGB_flight_data_ordinal_layer1', 0.7319502357820823), ('tBuQ__LGB